# Backpropagation

This document will explain the code I used to attempt to implement the backpropagation machine learning algorithm.

## What is it?
Backpropagation is an algorithm that is used in neural networks to learn. It is a form of supervised learning where the user gives it training examples with features and a desired output. The machine can try to predict the output then it will use a mutivariable function in order to calculate the error of each weight and bias. It does this for each training example then it tries to find the minimum of this function (essentially finding out what weights and biases correspond to the lowest error)